In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import gensim
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, concatenate
from tensorflow.keras.layers import Embedding
from keras.utils import np_utils
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

In [3]:
# Load data
df = pd.read_csv('hate-by-Kabada.csv', encoding = 'utf-8')
#,count,hate_speech,offensive_language,neither,class,tweet
# Preprocess data

ParserError: Error tokenizing data. C error: Expected 1 fields in line 73, saw 2


In [ ]:
df.head()

In [3]:
dt_transformed = df[['class', 'tweet']]
y = (dt_transformed.iloc[:, :-1].values).ravel()
dt_transformed

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies


In [4]:
# Dividindo o df em treino e teste
df_train, df_test = train_test_split(dt_transformed, test_size = 0.10, random_state = 42, stratify=dt_transformed['class'])
df_train.shape, df_test.shape

((22304, 2), (2479, 2))

In [5]:
corpus = preprocessing(df_train['tweet'].values)
corpus.shape

NameError: name 'preprocessing' is not defined

In [ ]:
# treino e validação do corpus
c_train, c_vad, y_train, y_vad = train_test_split(corpus, df_train['class'], test_size = 0.10, random_state = 42, stratify=df_train['class'])
c_train.shape, c_vad.shape

In [ ]:
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(c_train)
sequences = tokenizer.texts_to_sequences(c_train)
tokenizer.fit_on_texts(c_vad)
sequences = tokenizer.texts_to_sequences(c_vad)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
texts = df['tweet'].values.tolist()
labels = df['class'].values.tolist()

In [ ]:
# Load GloVe word embeddings
embedding_dim = 100
word_vectors = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_vectors[word] = coefs

In [ ]:
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=max_sequence_length)


In [ ]:
y=pd.get_dummies(df['class']).values
#y=pd.get_dummies(data['Intent']).values


In [ ]:

# Create embedding matrix
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = word_vectors.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Load pre-trained Word2Vec model.
#model = gensim.models.Word2Vec.load('Aoword2vec.bin')
#w2v_model = gensim.models.KeyedVectors.load_word2vec_format('Aoword2vec.bin', binary=True)

# Define parameters
max_sequence_length = 300
num_classes = 3
embedding_dim = 100
num_filters = 32
dropout_prob = 0.5
hidden_dims = 64
lr = 0.01
batch_size = 128
num_epochs = 2

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=max_sequence_length)
labels = np_utils.to_categorical(np.asarray(labels))
# Split data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1)

In [ ]:
x_train = np.array(X_train)

In [ ]:
x_train.shape

In [ ]:
x_train

In [ ]:
x_test = np.array(X_test)


In [ ]:
x_test

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
y_train

In [ ]:
# Define CNN model
inputs = Input(shape=(max_sequence_length,), dtype='int32')
embedding = Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=True)(inputs)
conv_0 = Conv1D(num_filters, 3, activation='relu')(embedding)
maxpool_0 = MaxPooling1D(max_sequence_length - 3 + 1)(conv_0)
dropout_0 = Dropout(dropout_prob)(maxpool_0)
conv_1 = Conv1D(num_filters, 4, activation='relu')(embedding)
maxpool_1 = MaxPooling1D(max_sequence_length - 4 + 1)(conv_1)
dropout_1 = Dropout(dropout_prob)(maxpool_1)
conv_2 = Conv1D(num_filters, 5, activation='relu')(embedding)
maxpool_2 = MaxPooling1D(max_sequence_length - 5 + 1)(conv_2)
dropout_2 = Dropout(dropout_prob)(maxpool_2)
merged_tensor = concatenate([dropout_0, dropout_1, dropout_2], axis=1)
flatten = Flatten()(merged_tensor)
dense = Dense(hidden_dims, activation='relu')(flatten)
dropout = Dropout(dropout_prob)(dense)
output = Dense(num_classes, activation='sigmoid')(dropout)
cnnmodel = Model(inputs, output)

In [154]:
# Compile CNN model
cnnmodel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnnmodel.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 300, 100)     1434800     ['input_4[0][0]']                
                                                                                                  
 conv1d_9 (Conv1D)              (None, 298, 32)      9632        ['embedding_3[0][0]']            
                                                                                                  
 conv1d_10 (Conv1D)             (None, 297, 32)      12832       ['embedding_3[0][0]']            
                                                                                            

In [117]:
# Train CNN model
cnnmodel.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs) #validation_data=(X_test, y_test))

Epoch 1/2
175/175 [==============================] - 47s 271ms/step - loss: 0.3395 - accuracy: 0.8876
Epoch 2/2
175/175 [==============================] - 47s 270ms/step - loss: 0.2892 - accuracy: 0.9068


In [118]:

# Extract features using CNN model
get_dense_layer_output = Model(model.input, model.layers[-3].output)
X_train_features = get_dense_layer_output.predict(x_train, verbose=1)
X_test_features = get_dense_layer_output.predict(x_test, verbose=1)

AttributeError: 'LogisticRegression' object has no attribute 'input'

In [ ]:
# Train LR model on extracted features
#clf = LogisticRegression(C=lr, penalty='l2')

In [ ]:
#clf.fit(x_train, y_train.ravel())
#y_pred = model.predict(x_test)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
#X_train = vectorizer.fit_transform(x_train)
#X_test = vectorizer.transform(x_test)

# Train Logistic Regression model
#lr_model = LogisticRegression()
#lr_model.fit(x_train, y_train)

In [ ]:
#lr_model = LogisticRegression()
# Train LR model
#lr_model.fit(X_train, y_train)

In [119]:
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer

In [120]:
dt_transformed = df[['class', 'tweet']]
y = (dt_transformed.iloc[:, :-1].values).ravel()
dt_transformed

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,2,"you've gone and broke the wrong heart baby, an..."
24780,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,1,youu got wild bitches tellin you lies


In [121]:
# Dividindo o df em treino e teste
df_train, df_test = train_test_split(dt_transformed, test_size = 0.10, random_state = 42, stratify=dt_transformed['class'])
df_train.shape, df_test.shape

((22304, 2), (2479, 2))

In [122]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [123]:
def preprocessing(data):
    stemmer = nltk.stem.RSLPStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    corpus = []
    for tweet in data:
        review = re.sub(r"@[A-Za-z0-9_]+", " ", tweet)
        review = re.sub('RT', ' ', review)
        review = re.sub(r"https?://[A-Za-z0-9./]+", " ", review)
        review = re.sub(r"https?", " ", review)
        review = re.sub('[^a-zA-Z]', ' ', review)
        review = review.lower()
        review = review.split()
        ps = PorterStemmer()
        review = [ps.stem(word) for word in review if not word in set(all_stopwords) if len(word) > 2]
        review = ' '.join(review)
        corpus.append(review)
    return np.array(corpus)

In [124]:
corpus = preprocessing(df_train['tweet'].values)
corpus.shape

(22304,)

In [125]:
# treino e validação do corpus
c_train, c_vad, y_train, y_vad = train_test_split(corpus, df_train['class'], test_size = 0.10, random_state = 42, stratify=df_train['class'])
c_train.shape, c_vad.shape

((20073,), (2231,))

In [126]:
def tokenize(c_train, c_vad):
    tweet_tokenizer = TweetTokenizer() 
    vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize, max_features = 1010)
    X_train = vectorizer.fit_transform(c_train).toarray()
    X_vad = vectorizer.transform(c_vad).toarray()
    return X_train, X_vad

In [127]:
X_train, X_vad = tokenize(c_train, c_vad)
X_train.shape, X_vad.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


((20073, 1010), (2231, 1010))

In [128]:
# Logistic Regression
lr = LogisticRegression(multi_class='ovr', solver='liblinear', random_state = 0)
lr.fit(X_train, y_train.ravel())
y_pred = model.predict(X_vad)

In [129]:
# Ensemble models
ensemble_preds = []
for i in range(2231):
    cnn_pred = cnn_model.predict(np.array([test_data[i]]))[0][0]
    lr_pred = lr_model.predict_proba(np.array([test_data[i]]))[0][1]
    ensemble_pred = (cnn_pred + lr_pred) / 2
    ensemble_preds.append(ensemble_pred)

# Evaluate ensemble model
ensemble_preds = np.array(ensemble_preds)
ensemble_preds[ensemble_preds >= 0.5] = 1
ensemble_preds[ensemble_preds < 0.5] = 0
ensemble_acc = accuracy_score(test_labels, ensemble_preds)
print('Ensemble model accuracy:', ensemble_acc)

NameError: name 'cnn_model' is not defined

In [ ]:
y_pred = clf.predict(X_test_features)
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
print('Accuracy:', accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:
y_train_shaped = np.ravel(y_train) 
x_train_shaped = np.ravel(X_train_features)
clf.fit(X_train_features, y_train_shaped)